# GA DSI 26: Project 2 - Ames Housing Data and Kaggle Challenge
***

### Project Notebooks organisation  
- Notebook 1 - Data Cleaning, EDA and Data Visualisation
- **Notebook 2 - Feature Engineering (current notebook)**
- Notebook 3 - Pre-processing, Model Benchmarks, Production Model and Kaggle Submission

***

## Introduction

In this notebook, I will be covering the feature engineering of the columns which will be mentioned below to create new features from existing features within the original `train` dataset. 

In [1]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('ticks')

In [2]:
# import cleaned train data

train_clean  = pd.read_csv('../datasets/train_clean.csv')

In [3]:
# to change ms_subclass type back to str after importing

train_clean['ms_subclass'] = train_clean['ms_subclass'].astype(str)

### Feature Engineering
***
As seen from Notebook 1 and these are the columns that are identified from the EDA that could use feature engineering to help reduce the number of features or to combine those features that are able to do so. 

<u>Columns to change or combine: </u>
- Combine `'3ssn_porch', 'enclosed_porch', 'open_porch_sf', 'screen_porch'` into a single column called `'porch'` to indicate a presence of a porch.
- A new column called `'age_sold'` will be created which will be obtained by deducting `'year_built'` from `'yr_sold'` to get the age of the house when it was sold. The `'yr_sold'` and `'year_built'` columns would then be dropped.
- `'pool_area'` will be change to a `'pool'` column to indicate if the house has a pool as a pool should highly affect the sale price of a house even though there are a lot of single values in this column.
- `'year_remod/add'` to change to `'remodelled'` if the house has been remodelled if this value is different from the `'year_built'` column.
- Change `'exterior_1st'` column, renaming it to `exterior` and reorganising the values into more general categories and dropping `'exterior_2nd'` column as the values are similar to `'exterior_1st'` column.
- Convert `'ms_subclass'` into categorical class
- Changing the ordinal features into numerical categories where 0 will be lowest of the scale and the largest number will be highest of the scale (depending on the scale) These columns include `'exter_qual', 'bsmtfin_type_1', 'heating_qc', 'kitchen_qual', 'fireplace_qu', 'bsmt_qual', 'lot_shape'`.

In [4]:
# define a lambda function to create 1 if the feature is present and 0 if the feature is not present

present = lambda x : 1 if x > 0 else 0

#### Porch Column

I will be creating a new ```'porch'``` column to indicate the presence of a porch by putting the value 1 if there are any values in the ```'3ssn_porch', 'enclosed_porch', 'open_porch_sf', 'screen_porch'``` columns and value 0 if there are no values in these columns.

Afterwards, i will be dropping all these columns (```'3ssn_porch', 'enclosed_porch', 'open_porch_sf', 'screen_porch'```).

In [5]:
porch_col = ['3ssn_porch', 'enclosed_porch', 'open_porch_sf', 'screen_porch']

train_clean['porch'] = train_clean[porch_col].sum(axis = 1).apply(present)
train_clean.drop(porch_col, axis = 1, inplace = True)

#### Pool Column

A new column called ```'pool'``` will be created to indicate the presence of a pool in the house by putting the value 1 if a pool is present and 0 if there is no pool. The ```'pool_area'``` column would then be dropped.


In [6]:
train_clean['pool'] = train_clean['pool_area'].apply(present)
train_clean.drop(['pool_area'], axis = 1, inplace = True)

#### Age Sold Column

A new column called ```'age_sold'``` will be created which will be obtained by deducting ```'year_built'``` from ```'yr_sold'``` to get the age of the house when it was sold. The ```'yr_sold'``` and ```'year_built'``` columns would then be dropped.


In [7]:
train_clean['age_sold'] = train_clean['yr_sold'] - train_clean['year_built']

# check the values of the 'age_sold' column
train_clean['age_sold'].unique()

array([ 34,  13,  57,   4, 110,  44,   1,  51,  55,  38,  37, 129,   9,
         6,  92,  46,  30,  39,  18,  48,  45,  71, 112,  58,  28,  77,
        11,   2,  29,  33,  43,   0,  31,  54,  73,  68,  56,   5,  81,
        40,  41,  52,  12,  88,  47,  78,  82,  50,  97,  93,  15,  49,
        87,  32, 109,   8,  91,   7,  70,  62,  53,  83,  74,  35, 135,
        98,  59,  76,  27,  95,  19,  36,  89,  69,  24,  65, 118,  66,
         3,  42, 106,  10,  20,  22,  99,  26,  23,  60,  25,  86,  90,
        21,  84,  14,  63,  94,  72,  67, 125, 114,  61,  96,  64,  17,
       119,  16,  80, 100, 107,  79, 101, 108, 127,  85, 136, 103, 128,
       115, 117, 111,  75, 113], dtype=int64)

There is an odd value where the age sold of the house is -1 which is not possible. Hence, we will be dropping that row.

In [8]:
# dropping of row with -1 in age_sold column

train_clean.drop(train_clean[train_clean['age_sold'] == -1].index, inplace = True)

#### Remodelled Column

Create a new column called ```'remodelled'``` where it will indicate if the house has been remodelled. If the value in ```'year_remod/add'``` is different from the ```'year_built'``` column, it will indicate a value 1 for presence of remodelling and 0 for no remodelling has been done.

In [9]:
train_clean['remodelled'] = train_clean['year_remod/add'] - train_clean['year_built']

train_clean['remodelled'] = train_clean['remodelled'].apply(present)

train_clean['remodelled'].value_counts()

0    1100
1     942
Name: remodelled, dtype: int64

In [10]:
# drop the 'year_remod/add', 'year_built' columns

train_clean.drop(['year_remod/add', 'year_built', 'yr_sold'], axis = 1, inplace = True)

#### Exterior Materials Column

From the comparison between the ```'exterior_1st'``` and ```'exterior_2nd'``` below and from the data dictionary from Kaggle, it can be seen that the materials in the houses are pretty similar where there are some different spelling of the materials used like ```CemntBd``` in ```'exterior_1st'``` and ```CmentBd``` in ```'exterior_2nd'```. Hence, I would be using ```'exterior_1st'``` column, renaming it to ```exterior``` and reorganising the values into more general categories like wood, metal, brick, stone, vinyl and cement aggregate.

In [11]:
# Check the values inside 'exterior_1st' and 'exterior_2nd' column

print(train_clean['exterior_1st'].value_counts())
print()
print(train_clean['exterior_2nd'].value_counts())

VinylSd    724
MetalSd    330
HdBoard    299
Wd Sdng    273
Plywood    150
CemntBd     89
BrkFace     64
WdShing     45
AsbShng     33
Stucco      26
BrkComm      3
CBlock       2
Stone        2
ImStucc      1
AsphShn      1
Name: exterior_1st, dtype: int64

VinylSd    721
MetalSd    324
HdBoard    274
Wd Sdng    261
Plywood    181
CmentBd     89
Wd Shng     63
BrkFace     34
Stucco      29
AsbShng     28
Brk Cmn     17
ImStucc     11
Stone        6
AsphShn      3
CBlock       1
Name: exterior_2nd, dtype: int64


In [12]:
# Define a dictionary to map the values from 'exterior_1st' columns into more general 
# categories in the new 'exterior' column

exterior_materials = {'HdBoard' : 'wood', 
                      'VinylSd' : 'vinyl', 
                      'Wd Sdng' : 'wood', 
                      'BrkFace' : 'brick', 
                      'Plywood' : 'wood',
                      'MetalSd' : 'metal',
                      'AsbShng' : 'cemt_agg',
                      'CemntBd' : 'cemt_agg',
                      'WdShing': 'wood',
                      'Stucco' : 'cemt_agg',
                      'BrkComm' : 'brick',
                      'Stone' : 'stone',
                      'CBlock' : 'cemt_agg',
                      'ImStucc' : 'cemt_agg',
                      'AsphShn' : 'cemt_agg'}

# Mapping the dictionary to create the new 'exterior' column
train_clean['exterior'] = train_clean['exterior_1st'].map(exterior_materials)

# Dropping 'exterior_1st' , 'exterior_2nd' columns
train_clean.drop(['exterior_1st' , 'exterior_2nd'], axis = 1, inplace =True)

#### Changing the ordinal features into numerical categories 

In [13]:
# function to map new values
def ord_to_num(col, dic, df):
    df[col] = df[col].map(dic)
    
# defining dictionaries for renaming
lotshape_dict = {'Reg' : 0,
                 'IR1' : 1,
                 'IR2' : 2,
                 'IR3' : 3}
    
quality_dict = {'Ex' : 5,
                'Gd' : 4,
                'TA' : 3,
                'Fa' : 2,
                'Po' : 1,
                'None' : 0}

bsmt_fin_dict = {'GLQ' : 6,
                 'ALQ' : 5,
                 'BLQ' : 4,
                 'Rec' : 3,
                 'LwQ' : 2,
                 'Unf' : 1,
                 'None' : 0}

# Changing the ordinal features into numerical categories
ord_to_num('lot_shape', lotshape_dict, train_clean)
ord_to_num('bsmt_qual', quality_dict, train_clean)
ord_to_num('heating_qc', quality_dict, train_clean)
ord_to_num('exter_qual', quality_dict, train_clean)
ord_to_num('fireplace_qu', quality_dict, train_clean)
ord_to_num('kitchen_qual', quality_dict, train_clean)
ord_to_num('bsmtfin_type_1', bsmt_fin_dict, train_clean)


In [14]:
# to check the datatypes of the train_clean dataset

train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2042 entries, 0 to 2041
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   1st_flr_sf      2042 non-null   int64  
 1   bsmt_exposure   2042 non-null   object 
 2   bsmt_full_bath  2042 non-null   float64
 3   bsmt_qual       2042 non-null   int64  
 4   bsmtfin_type_1  2042 non-null   int64  
 5   exter_qual      2042 non-null   int64  
 6   fireplace_qu    2042 non-null   int64  
 7   fireplaces      2042 non-null   int64  
 8   foundation      2042 non-null   object 
 9   full_bath       2042 non-null   int64  
 10  garage_area     2042 non-null   float64
 11  garage_finish   2042 non-null   object 
 12  garage_type     2042 non-null   object 
 13  garage_yr_blt   2042 non-null   float64
 14  gr_liv_area     2042 non-null   int64  
 15  half_bath       2042 non-null   int64  
 16  heating_qc      2042 non-null   int64  
 17  house_style     2042 non-null   o

In [15]:
# to check all feature engineering process have been carried out successfully 

train_clean.describe().T

count           mean           std      min        25%  \
1st_flr_sf      2042.0    1158.779138    374.138753    334.0     879.00   
bsmt_full_bath  2042.0       0.425563      0.520612      0.0       0.00   
bsmt_qual       2042.0       3.490695      0.895392      0.0       3.00   
bsmtfin_type_1  2042.0       3.561704      2.123655      0.0       1.00   
exter_qual      2042.0       3.405485      0.586580      2.0       3.00   
fireplace_qu    2042.0       1.759060      1.806876      0.0       0.00   
fireplaces      2042.0       0.587169      0.634229      0.0       0.00   
full_bath       2042.0       1.575416      0.548878      0.0       1.00   
garage_area     2042.0     472.766895    214.654611      0.0     319.00   
garage_yr_blt   2042.0    1869.231146    453.200585      0.0    1957.00   
gr_liv_area     2042.0    1493.855044    483.202953    334.0    1128.00   
half_bath       2042.0       0.371205      0.501163      0.0       0.00   
heating_qc      2042.0       4.159158      0.964148      1.0       3.00   
id              2042.0    1472.447600    845.088850      1.0     751.25   
kitchen_qual    2042.0       3.514691      0.664070      2.0       3.00   
lot_area        2042.0    9873.066601   5163.579574   1300.0    7500.00   
lot_frontage    2042.0      68.586680     19.845967     21.0      60.00   
lot_shape       2042.0       0.399608      0.560814      0.0       0.00   
mas_vnr_area    2042.0      97.668952    171.872197      0.0       0.00   
overall_qual    2042.0       6.109696      1.422750      1.0       5.00   
saleprice       2042.0  181354.994613  79279.035715  12789.0  129575.00   
total_bsmt_sf   2042.0    1053.817336    425.815748      0.0     793.00   
wood_deck_sf    2042.0      92.906464    124.748887      0.0       0.00   
porch           2042.0       0.701273      0.457812      0.0       0.00   
pool            2042.0       0.003918      0.062484      0.0       0.00   
age_sold        2042.0      36.050930     30.227722      0.0       7.00   
remodelled      2042.0       0.461312      0.498623      0.0       0.00   

                     50%        75%       max  
1st_flr_sf        1092.0    1402.00    3228.0  
bsmt_full_bath       0.0       1.00       3.0  
bsmt_qual            4.0       4.00       5.0  
bsmtfin_type_1       4.0       6.00       6.0  
exter_qual           3.0       4.00       5.0  
fireplace_qu         1.0       4.00       5.0  
fireplaces           1.0       1.00       4.0  
full_bath            2.0       2.00       4.0  
garage_area        480.0     576.00    1356.0  
garage_yr_blt     1978.0    2001.00    2207.0  
gr_liv_area       1442.0    1728.00    3672.0  
half_bath            0.0       1.00       2.0  
heating_qc           5.0       5.00       5.0  
id                1479.5    2198.50    2930.0  
kitchen_qual         3.0       4.00       5.0  
lot_area          9400.0   11488.75   70761.0  
lot_frontage        68.0      78.00     200.0  
lot_shape            0.0       1.00       3.0  
mas_vnr_area         0.0     158.00    1600.0  
overall_qual         6.0       7.00      10.0  
saleprice       162250.0  214000.00  611657.0  
total_bsmt_sf      994.0    1316.75    3206.0  
wood_deck_sf         0.0     168.00     870.0  
porch                1.0       1.00       1.0  
pool                 0.0       0.00       1.0  
age_sold            34.0      54.00     136.0  
remodelled           0.0       1.00       1.0

The ordinal features have been converted into numerical categories as seen from the summary statistics above where the data types of those columns have become int64 and the minimum and maximum ranges have also been revised.

### Feature engineering for test_clean dataset

After feature engineering has been completed for train_clean dataset, I will carry out the same steps for the test_clean dataset.

In [16]:
# import test_clean data set

test_clean = pd.read_csv('../datasets/test_clean.csv')

# change ms_subclass datatype to str after import
test_clean['ms_subclass'] = test_clean['ms_subclass'].astype(str)

In [17]:
test_clean.columns

Index(['id', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'lot_shape', 'lot_config', 'neighborhood', 'house_style',
       'overall_qual', 'year_built', 'year_remod/add', 'roof_style',
       'exterior_1st', 'exterior_2nd', 'mas_vnr_type', 'mas_vnr_area',
       'exter_qual', 'foundation', 'bsmt_qual', 'bsmt_exposure',
       'bsmtfin_type_1', 'total_bsmt_sf', 'heating_qc', '1st_flr_sf',
       'gr_liv_area', 'bsmt_full_bath', 'full_bath', 'half_bath',
       'kitchen_qual', 'fireplaces', 'fireplace_qu', 'garage_type',
       'garage_yr_blt', 'garage_finish', 'garage_area', 'wood_deck_sf',
       'open_porch_sf', 'enclosed_porch', '3ssn_porch', 'screen_porch',
       'pool_area', 'yr_sold'],
      dtype='object')

To check if the values in ```exterior_1st``` in the ```test_clean``` dataset is the same as the ```train``` dataset in order for the dictionary mapping to work

In [18]:
test_clean['exterior_1st'].value_counts()

VinylSd    302
Wd Sdng    144
HdBoard    142
MetalSd    119
Plywood     69
CemntBd     35
BrkFace     24
Stucco      16
AsbShng     11
WdShing     11
BrkComm      3
PreCast      1
AsphShn      1
Name: exterior_1st, dtype: int64

In the ```test_clean``` dataset, there is an additional label of ```PreCast``` that is no present in the ```train_clean``` dataset for ```exterior_1st``` column. Hence, I will be adding another key, value pair in the exterior_materials dictionary in the function below to change the ```PreCast``` label to ```cemt_agg```.

In [19]:
def feature_eng(df):
    """function to carry out feature engineering tasks for dataframe"""
    
    # create new porch column to indicate the presence of a porch
    porch_col = ['3ssn_porch', 'enclosed_porch', 'open_porch_sf', 'screen_porch']
    df['porch'] = df[porch_col].sum(axis = 1).apply(present)
    df.drop(porch_col, axis = 1, inplace = True)

    # create pool column to indicate presence of a pool
    df['pool'] = df['pool_area'].apply(present)
    df.drop(['pool_area'], axis = 1, inplace = True)
    
    # create age_sold column
    df['age_sold'] = df['yr_sold'] - df['year_built']

    # create remodelled column to indicate if a house has been remodelled after it was built
    df['remodelled'] = df['year_remod/add'] - df['year_built']
    df['remodelled'] = df['remodelled'].apply(present)

    # creating new exterior column and reorganising the labels
    exterior_materials = {'HdBoard' : 'wood', 
                          'VinylSd' : 'vinyl', 
                          'Wd Sdng' : 'wood', 
                          'BrkFace' : 'brick', 
                          'Plywood' : 'wood',
                          'MetalSd' : 'metal',
                          'AsbShng' : 'cemt_agg',
                          'CemntBd' : 'cemt_agg',
                          'WdShing': 'wood',
                          'Stucco' : 'cemt_agg',
                          'BrkComm' : 'brick',
                          'Stone' : 'stone',
                          'CBlock' : 'cemt_agg',
                          'ImStucc' : 'cemt_agg',
                          'AsphShn' : 'cemt_agg',
                          'PreCast': 'cemt_agg'}
    df['exterior'] = df['exterior_1st'].map(exterior_materials)

    # defining dictionaries for renaming
    lotshape_dict = {'Reg' : 0,
                     'IR1' : 1,
                     'IR2' : 2,
                     'IR3' : 3}

    quality_dict = {'Ex' : 5,
                    'Gd' : 4,
                    'TA' : 3,
                    'Fa' : 2,
                    'Po' : 1,
                    'None' : 0}

    bsmt_fin_dict = {'GLQ' : 6,
                     'ALQ' : 5,
                     'BLQ' : 4,
                     'Rec' : 3,
                     'LwQ' : 2,
                     'Unf' : 1,
                     'None' : 0}

    # Changing the ordinal features into numerical categories
    ord_to_num('lot_shape', lotshape_dict, df)
    ord_to_num('bsmt_qual', quality_dict, df)
    ord_to_num('heating_qc', quality_dict, df)
    ord_to_num('exter_qual', quality_dict, df)
    ord_to_num('fireplace_qu', quality_dict, df)
    ord_to_num('kitchen_qual', quality_dict, df)
    ord_to_num('bsmtfin_type_1', bsmt_fin_dict, df)

    # drop all the columns that are no longer needed after feature engineering
    df.drop(['year_remod/add', 'year_built', 'yr_sold', 'exterior_1st' , 'exterior_2nd'], axis = 1, inplace = True)
    
    return df

In [20]:
feature_eng(test_clean)

id ms_subclass ms_zoning  lot_frontage  lot_area  lot_shape lot_config  \
0    2658         190        RM          69.0      9142          0     Inside   
1    2718          90        RL          68.0      9662          1     Inside   
2    2414          60        RL          58.0     17104          1     Inside   
3    1989          30        RM          60.0      8520          0     Inside   
4     625          20        RL          68.0      9500          1     Inside   
..    ...         ...       ...           ...       ...        ...        ...   
873  1662          60        RL          80.0      8000          0     Inside   
874  1234          60        RL          90.0     14670          0     Inside   
875  1373          20        RL          55.0      8250          0     Inside   
876  1672          20        RL          60.0      9000          0        FR2   
877  1939          20        RL          70.0      8400          0     Corner   

    neighborhood house_style  overall_qual  ... garage_type garage_yr_blt  \
0        OldTown      2Story             6  ...      Detchd        1910.0   
1         Sawyer      1Story             5  ...      Attchd        1977.0   
2        Gilbert      2Story             7  ...      Attchd        2006.0   
3        OldTown      1Story             5  ...      Detchd        1935.0   
4          NAmes      1Story             6  ...      Attchd        1963.0   
..           ...         ...           ...  ...         ...           ...   
873       NWAmes      2Story             6  ...      Attchd        1974.0   
874        NAmes      2Story             6  ...      Attchd        1966.0   
875       Sawyer      1Story             5  ...      Attchd        1968.0   
876        NAmes      1Story             4  ...      Detchd        1974.0   
877        NAmes      1Story             5  ...      Attchd        1955.0   

     garage_finish  garage_area wood_deck_sf  porch pool  age_sold  \
0              Unf          440            0      1    0        96   
1              Fin          580          170      0    0        29   
2              RFn          426          100      1    0         0   
3              Unf          480            0      1    0        84   
4              RFn          514            0      1    0        46   
..             ...          ...          ...    ...  ...       ...   
873            Unf          488            0      1    0        33   
874            RFn          480            0      1    0        42   
875            Unf          322            0      1    0        40   
876            Unf          528            0      0    0        36   
877            Unf          390           81      0    0        52   

     remodelled  exterior  
0             1  cemt_agg  
1             0      wood  
2             0     vinyl  
3             1      wood  
4             0      wood  
..          ...       ...  
873           0      wood  
874           1     vinyl  
875           0      wood  
876           0      wood  
877           0     metal  

[878 rows x 38 columns]

### Output feature engineered data for train and test dataset

These datasets will be used in the next notebook for pre-processing, modelling and Kaggle submission.

In [21]:
train_eng = train_clean.copy()
train_eng.to_csv('../datasets/train_eng.csv')
test_eng = test_clean.copy()
test_eng.to_csv('../datasets/test_eng.csv')